# NEWSPAPER TITLE CLASSIFICATION BASED ON KNN, KMEANS AND DECISION TREE

To have general view & data structure of Project, refer the `Readme.md` of this Project and general structure of project

![General Structure](general_structure.png)

## 1. IMPORT LIBRARY

In [8]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from langchain.evaluation import load_dataset

In [9]:
import re
import nltk # use in case

In [10]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix

## 2. DATA FEATURE EXTRACTION

`Note: Refer the Readme.md for Data source collection`

In [11]:
from datasets import load_dataset
ds = load_dataset('UniverseTBD/arxiv-abstracts-large')

Dataset is dictionary collection with feature fields

In [14]:
print(type(ds))
ds

<class 'datasets.dataset_dict.DatasetDict'>


DatasetDict({
    train: Dataset({
        features: ['id', 'submitter', 'authors', 'title', 'comments', 'journal-ref', 'doi', 'report-no', 'categories', 'license', 'abstract', 'versions', 'update_date', 'authors_parsed'],
        num_rows: 2292057
    })
})

Find the path of the data files

The data has been downloaded then stored in the path (with Linux)
"/home/anhvt/.cache/huggingface/datasets/UniverseTBD___arxiv-abstracts-large/default/*.arrow"

During loading wiht dataset, the command load_dataset shall be removed to cancel the download program

In [15]:
# list down the path of data file
print(ds.cache_files)

{'train': [{'filename': '/home/anhvt/.cache/huggingface/datasets/UniverseTBD___arxiv-abstracts-large/default/0.0.0/6020a62078a73d7ca02b86a4a775af7caba42d5e/arxiv-abstracts-large-train-00000-of-00007.arrow'}, {'filename': '/home/anhvt/.cache/huggingface/datasets/UniverseTBD___arxiv-abstracts-large/default/0.0.0/6020a62078a73d7ca02b86a4a775af7caba42d5e/arxiv-abstracts-large-train-00001-of-00007.arrow'}, {'filename': '/home/anhvt/.cache/huggingface/datasets/UniverseTBD___arxiv-abstracts-large/default/0.0.0/6020a62078a73d7ca02b86a4a775af7caba42d5e/arxiv-abstracts-large-train-00002-of-00007.arrow'}, {'filename': '/home/anhvt/.cache/huggingface/datasets/UniverseTBD___arxiv-abstracts-large/default/0.0.0/6020a62078a73d7ca02b86a4a775af7caba42d5e/arxiv-abstracts-large-train-00003-of-00007.arrow'}, {'filename': '/home/anhvt/.cache/huggingface/datasets/UniverseTBD___arxiv-abstracts-large/default/0.0.0/6020a62078a73d7ca02b86a4a775af7caba42d5e/arxiv-abstracts-large-train-00004-of-00007.arrow'}, {'fi

Collect the fields by keys of dict dataset

In [20]:
train = ds['train']
topic_features = train.column_names
print(topic_features)

['id', 'submitter', 'authors', 'title', 'comments', 'journal-ref', 'doi', 'report-no', 'categories', 'license', 'abstract', 'versions', 'update_date', 'authors_parsed']
